# Using LSTMs with CNN based embeddings for Utterances to Phoneme Mapping

### Author: Lakshay Sethi
#### Custom Architecture 3 (Designed by Lakshay Sethi through Ablations)
##### Ablations Link: https://wandb.ai/verydeeplearning/hw3p2-ablations

In [ ]:
!nvidia-smi

Fri Apr  7 08:26:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install wandb

### Levenshtein

This may take a while

In [ ]:
# !pip install wandb --quiet
# !pip install python-Levenshtein -q
# !git clone --recursive https://github.com/parlance/ctcdecode.git
# !pip install wget -q
# %cd ctcdecode
# !pip install . -q
# %cd ..

# !pip install torchsummaryX -q

## Imports

In [ ]:
import torch
import torchaudio
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"lakshaysethi","key":"32d86595e55bef36c1f649381dc3282f"}')

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# !kaggle competitions download -c 11-785-s23-hw3p2
# '''
# This will take a couple minutes, but you should see at least the following:
# 11-785-f22-hw3p2.zip  ctcdecode  hw3p2
# '''
# !unzip -q 11-785-s23-hw3p2.zip
# !ls

# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
PHONEMES

['',
 '[SIL]',
 'NG',
 'F',
 'M',
 'AE',
 'R',
 'UW',
 'N',
 'IY',
 'AW',
 'V',
 'UH',
 'OW',
 'AA',
 'ER',
 'HH',
 'Z',
 'K',
 'CH',
 'W',
 'EY',
 'ZH',
 'T',
 'EH',
 'Y',
 'AH',
 'B',
 'P',
 'TH',
 'DH',
 'AO',
 'G',
 'L',
 'JH',
 'OY',
 'SH',
 'D',
 'AY',
 'S',
 'IH']

In [ ]:
ARPAbet

[' ',
 '-',
 'G',
 'f',
 'm',
 '@',
 'r',
 'u',
 'n',
 'i',
 'W',
 'v',
 'U',
 'o',
 'a',
 'R',
 'h',
 'z',
 'k',
 'C',
 'w',
 'e',
 'Z',
 't',
 'E',
 'y',
 'A',
 'b',
 'p',
 'T',
 'D',
 'c',
 'g',
 'l',
 'j',
 'O',
 'S',
 'd',
 'Y',
 's',
 'I',
 '[SOS]',
 '[EOS]']

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, partition = ['train-clean-100']):
        self.mfcc_files, self.transcript_files = [], []

        for part in partition:  
            mfcc_dir = root + part + '/mfcc/'
            transcript_dir = root + part +'/transcript/'

            mfcc_temp_dir = os.listdir(mfcc_dir)
            mfcc_temp_dir.sort()
            mfcc_temp = [os.path.join(mfcc_dir, currFile) for currFile in mfcc_temp_dir]
            self.mfcc_files.extend(mfcc_temp)

            transcript_temp_dir = os.listdir(transcript_dir)
            transcript_temp_dir.sort()
            transcript_temp = [os.path.join(transcript_dir, currFile) for currFile in transcript_temp_dir]
            self.transcript_files.extend(transcript_temp)

        
        # Sanity Check
        assert len(self.mfcc_files) == len(self.transcript_files)

        # Lenght Assignment
        self.length = len(self.mfcc_files)

        self.PHONEMES = phonemes
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        self.PHONEMES_DICT = {value: idx for idx, value in enumerate(self.PHONEMES)}


        self.mfccs, self.transcripts = [], []

        for i in range(self.length):
            mfcc = np.load(self.mfcc_files[i], allow_pickle=True)
            mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
            transcript = np.load(self.transcript_files[i], allow_pickle=True)
            transcript = transcript[1:-1]         # Removing [SOS] and [EOS] from the transcript 

            label = np.array([self.PHONEMES_DICT[trans] for trans in transcript])

            self.mfccs.append(mfcc)
            self.transcripts.append(label)
    

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc_tensor = torch.FloatTensor(self.mfccs[ind])
        transcript_tensor = torch.tensor(self.transcripts[ind])

        return mfcc_tensor, transcript_tensor


    def collate_fn(self, batch):
        batch_mfcc       = [mfcc for mfcc, _ in batch]
        batch_transcript = [transcript for _, transcript in batch]

        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0)
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True, padding_value=0.0)
        lengths_transcript = [len(trans) for trans in batch_transcript]

        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

In [ ]:
# class customCollate:

#     def __init__(self, trainData = True):
#         self.trainData = trainData

#     def __call__(self, batch):
#         batch_mfcc       = [mfcc for mfcc, _ in batch]
#         batch_transcript = [transcript for _, transcript in batch]

#         if self.trainData:
#             time_masking = torchaudio.transforms.TimeMasking(time_mask_param = 10, iid_masks = False, p = 1.0)
#             frequency_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param = 20, iid_masks = False)
            
#             batch_mfcc = 

#         batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0)
#         lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc]

#         batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True, padding_value=0.0)
#         lengths_transcript = [len(trans) for trans in batch_transcript]

#         return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

### Test Data

In [ ]:
# Test Dataloader
class AudioDatasetTest(torch.utils.data.Dataset):

    def __init__(self, root, partition = 'test-clean'): 

        self.mfcc_dir = root + partition + '/mfcc/'
        self.mfcc_files = os.listdir(self.mfcc_dir)
        self.mfcc_files.sort()

        self.length = len(self.mfcc_files)

        self.mfccs = []

        for i in range(self.length):
            mfcc = np.load(self.mfcc_dir + self.mfcc_files[i], allow_pickle=True)
            mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
            self.mfccs.append(mfcc)
    
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc = torch.FloatTensor(self.mfccs[ind])
        return mfcc

    def collate_fn(self,batch):
        batch_mfcc = batch
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True, padding_value=0)
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc]

        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 240 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2' 

### Data loaders

In [ ]:
gc.collect()

0

In [ ]:
# del train_data, val_data, test_data, train_loader, val_loader, test_loader

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(root = '/content/11-785-s23-hw3p2/', phonemes = PHONEMES, partition = ['train-clean-100', 'train-clean-360'])
val_data =AudioDataset(root = '/content/11-785-s23-hw3p2/', phonemes = PHONEMES, partition = ['dev-clean'])
test_data = AudioDatasetTest(root = '/content/11-785-s23-hw3p2/', partition = 'test-clean')

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = BATCH_SIZE, 
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)
val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = BATCH_SIZE,
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = val_data.collate_fn
)
test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = BATCH_SIZE, 
    pin_memory  = True, 
    shuffle     = False,
    collate_fn  = test_data.collate_fn
)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  240
Train dataset samples = 132552, batches = 553
Val dataset samples = 2703, batches = 12
Test dataset samples = 2620, batches = 11


In [ ]:
gc.collect()

1980

In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([240, 1679, 27]) torch.Size([240, 233]) torch.Size([240]) torch.Size([240])


In [ ]:
print(len(x))
print(len(lx))

240
240


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
# torch.cuda.empty_cache()

# class Network(nn.Module):

#     def __init__(self):

#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         self.embedding = nn.Sequential(nn.Conv1d(27, 512, kernel_size = 9, stride = 1, padding = 4, bias = False),
#                                        nn.GELU(),
#                                        nn.BatchNorm1d(512),
                                       
#                                        nn.Conv1d(512, 512, kernel_size = 7, stride = 1, padding = 3, bias = False),
#                                        nn.GELU(),
#                                        nn.BatchNorm1d(512),

#                                        nn.Conv1d(512, 512, kernel_size = 5, stride = 1, padding = 2, bias = False),
#                                        nn.GELU(),
#                                        nn.BatchNorm1d(512),

#                                        nn.Conv1d(512, 512, kernel_size = 3, stride = 1, padding = 1, bias = False),
#                                        nn.BatchNorm1d(512)
#         )

#         self.out_features = len(PHONEMES)
        
#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size = 512, hidden_size = 384, num_layers = 5, bias = True, bidirectional = True, dropout = 0.1) 
       
#         self.classification = nn.Sequential(
#             nn.Dropout(p = 0.1, inplace=False),
#             torch.nn.Linear(384*2, 384),
#             torch.nn.Linear(384, self.out_features)
#         )

#         self.logSoftmax = torch.nn.LogSoftmax(dim = 2)

#     def forward(self, x, lx):
#         x = self.embedding(torch.transpose(x, 1, 2))
#         x = torch.transpose(x, 1, 2)

#         x = pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)
#         x, _ = self.lstm(x)
#         del _

#         x, lens_unpacked  = pad_packed_sequence(x, batch_first=True)
#         x = self.classification(x)
#         x = F.log_softmax(x, dim=2)

#         return x, lens_unpacked

In [ ]:
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        self.embedding = nn.Sequential(PermuteBlock(),
                                       nn.Conv1d(27, 128, kernel_size = 3, stride = 2, padding = 1, bias = False),
                                       nn.Dropout(p = 0.3, inplace = False),
                                       nn.GELU(),
                                       nn.BatchNorm1d(128),
                                       
                                       nn.Conv1d(128, 256, kernel_size = 3, stride = 2, padding = 1, bias = False),
                                       nn.Dropout(p = 0.3, inplace = False),
                                       nn.GELU(),
                                       nn.BatchNorm1d(256),
                                       PermuteBlock()
        )

        self.out_features = len(PHONEMES)
        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm1 = nn.LSTM(input_size = 256, hidden_size = 384, num_layers = 3, bias = True, bidirectional = True, dropout = 0.2)
        self.lstm2 = nn.LSTM(input_size = 768, hidden_size = 768, num_layers = 2, bias = True, bidirectional = True, dropout = 0.2)
       
        self.classification = nn.Sequential(
            torch.nn.Linear(1536, 1536),
            nn.Dropout(p = 0.2, inplace = False),
            torch.nn.Linear(1536, self.out_features)
        )

        self.logSoftmax = torch.nn.LogSoftmax(dim = 2)

    def forward(self, x, lx):
        x = self.embedding(x)

        lx = (lx-2)//4
        x = pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)
        x, _ = self.lstm1(x)
        del _
        x, _ = self.lstm2(x)
        del _

        x, lens_unpacked  = pad_packed_sequence(x, batch_first=True)
        x = self.classification(x)
        x = F.log_softmax(x, dim=2)

        return x, lens_unpacked

# INIT
(If trying out the basic Network)

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                              Kernel Shape      Output Shape      Params  \
Layer                                                                      
0_embedding.PermuteBlock_0               -   [240, 27, 1679]           -   
1_embedding.Conv1d_1          [27, 128, 3]   [240, 128, 840]     10.368k   
2_embedding.Dropout_2                    -   [240, 128, 840]           -   
3_embedding.GELU_3                       -   [240, 128, 840]           -   
4_embedding.BatchNorm1d_4            [128]   [240, 128, 840]       256.0   
5_embedding.Conv1d_5         [128, 256, 3]   [240, 256, 420]     98.304k   
6_embedding.Dropout_6                    -   [240, 256, 420]           -   
7_embedding.GELU_7                       -   [240, 256, 420]           -   
8_embedding.BatchNorm1d_8            [256]   [240, 256, 420]       512.0   
9_embedding.PermuteBlock_9               -   [240, 420, 256]           -   
10_lstm1                                 -      [75977, 768]     9.0624M   
11_lstm2    

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding.PermuteBlock_0,-,"[240, 27, 1679]",NaN,NaN
1_embedding.Conv1d_1,"[27, 128, 3]","[240, 128, 840]",10368.0,8709120.0
2_embedding.Dropout_2,-,"[240, 128, 840]",NaN,NaN
3_embedding.GELU_3,-,"[240, 128, 840]",NaN,NaN
4_embedding.BatchNorm1d_4,[128],"[240, 128, 840]",256.0,128.0
5_embedding.Conv1d_5,"[128, 256, 3]","[240, 256, 420]",98304.0,41287680.0
6_embedding.Dropout_6,-,"[240, 256, 420]",NaN,NaN
7_embedding.GELU_7,-,"[240, 256, 420]",NaN,NaN
8_embedding.BatchNorm1d_8,[256],"[240, 256, 420]",512.0,256.0


In [10]:
!nvidia-smi

In [ ]:
torch.cuda.empty_cache()

In [9]:
!nvidia-smi

## Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# # Utils for network
# torch.cuda.empty_cache()

# class PermuteBlock(torch.nn.Module):
#     def forward(self, x):
#         return x.transpose(1, 2)

In [ ]:
# class pBLSTM(torch.nn.Module):

#     '''
#     Pyramidal BiLSTM
#     Read the write up/paper and understand the concepts and then write your implementation here.

#     At each step,
#     1. Pad your input if it is packed (Unpack it)
#     2. Reduce the input length dimension by concatenating feature dimension
#         (Tip: Write down the shapes and understand)
#         (i) How should  you deal with odd/even length input? 
#         (ii) How should you deal with input length array (x_lens) after truncating the input?
#     3. Pack your input
#     4. Pass it into LSTM layer

#     To make our implementation modular, we pass 1 layer at a time.
#     '''
    
#     def __init__(self, input_size, hidden_size):
#         super(pBLSTM, self).__init__()

#         self.blstm = # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

#     def forward(self, x_packed): # x_packed is a PackedSequence

#         # TODO: Pad Packed Sequence
        
#         # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
#         # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
#         # TODO: Pack Padded Sequence. What output(s) would you get?
#         # TODO: Pass the sequence through bLSTM

#         # What do you return?

#         return NotImplemented

#     def trunc_reshape(self, x, x_lens): 
#         # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
#         # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
#         # TODO: Reduce lengths by the same downsampling factor
#         return x, x_lens

# Encoder

In [ ]:
# class Encoder(torch.nn.Module):
#     '''
#     The Encoder takes utterances as inputs and returns latent feature representations
#     '''
#     def __init__(self, input_size, encoder_hidden_size):
#         super(Encoder, self).__init__()

        
#         self.embedding = #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.

#         self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
#             # TODO: Fill this up with pBLSTMs - What should the input_size be? 
#             # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
#             # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
#             # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
#             # ...
#             # ...
#         )
         
#     def forward(self, x, x_lens):
#         # Where are x and x_lens coming from? The dataloader
#         #TODO: Call the embedding layer
#         # TODO: Pack Padded Sequence
#         # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
#         # TODO: Pad Packed Sequence
        

#         # Remember the number of output(s) each function returns

#         return encoder_outputs, encoder_lens

# Decoder

In [ ]:
# class Decoder(torch.nn.Module):

#     def __init__(self, embed_size, output_size= 41):
#         super().__init__()

#         self.mlp = torch.nn.Sequential(
#             PermuteBlock(), torch.nn.BatchNorm1d(embed_size), PermuteBlock(),
#             #TODO define your MLP arch. Refer HW1P2
#             #Use Permute Block before and after BatchNorm1d() to match the size
#         )
        
#         self.softmax = torch.nn.LogSoftmax(dim=2)

#     def forward(self, encoder_out):
#         #TODO call your MLP
#         #TODO Think what should be the final output of the decoder for the classification 

#         return out 

In [ ]:
# class ASRModel(torch.nn.Module):

#     def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
#         super().__init__()

#         self.augmentations  = torch.nn.Sequential(
#             #TODO Add Time Masking/ Frequency Masking
#             #Hint: See how to use PermuteBlock() function defined above
#         )
#         self.encoder        =# TODO: Initialize Encoder
#         self.decoder        =# TODO: Initialize Decoder 

        
    
#     def forward(self, x, lengths_x):
        
#         if self.training:
#             x = self.augmentations(x)

#         encoder_out, encoder_lens   = self.encoder(x, lengths_x)
#         decoder_out                 = self.decoder(encoder_out)

#         return decoder_out, encoder_lens

## INIT ASR

In [ ]:
# model = ASRModel(
#     input_size  = 27,
#     embed_size  = #TODO
#     output_size = len(PHONEMES)
# ).to(device)
# print(model)
# torchsummaryX.summary(model, x.to(device), lx)

# Training Config

In [ ]:
config = {
    "beam_width" : 5,
    "lr" : 2e-3,
    "epochs" : 50
    }

In [8]:
criterion = torch.nn.CTCLoss()
# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr = config['lr'], weight_decay = config['lr']/100)

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(LABELS, beam_width = config['beam_width'], log_probs_input=True)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = config['epochs'], eta_min = config['lr']/200, verbose=True)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP = LABELS):
    
    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings    = []
    
    for i in range(beam_results.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        beam_temp = beam_results[i,0,:out_seq_len[i,0]]
        pred_str = "".join([PHONEME_MAP[i] for i in beam_temp])
        pred_strings.append(pred_str)
    
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP = LABELS): # y - sequence of integers
    
    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_str = pred_strings[i]
        label_temp = label[i,0:label_lens[i]]
        label_str = [PHONEME_MAP[k] for k in label_temp]
        dist += Levenshtein.distance(pred_str, label_str)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    # raise NotImplemented
    return dist

In [7]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    h = torch.permute(h, (1, 0, 2))
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

## wandb

You will need to fetch your api key from wandb.ai

In [5]:
import wandb
wandb.login(key="2a3537b39181a31bafb5eabede64d932c94e54e8")

In [6]:
run = wandb.init(
    name = "RNN_HW3_v2", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [4]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    h = torch.permute(h, (1, 0, 2))
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

### Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = 'Epoch_checkpoint_v2.pth'
best_model_path = 'Best_checkpoint_v2.pth'

In [3]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(25, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    
    curr_lr = float(optimizer.param_groups[0]['lr'])
    print('Current Learning rate: ', curr_lr)

    train_loss              = train_model(model, train_loader, criterion, optimizer)
    torch.cuda.empty_cache()
    gc.collect()
    
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, LABELS)
    torch.cuda.empty_cache()
    gc.collect()
    
    scheduler.step()
    #scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,  
        'valid_dist': valid_dist, 
        'valid_loss': valid_loss, 
        'lr'        : curr_lr
    })
    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
#run.finish()

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = int(config['beam_width']*1.5)

test_decoder    = CTCBeamDecoder(LABELS, beam_width = TEST_BEAM_WIDTH, log_probs_input=True)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string = decode_prediction(h, lh, test_decoder, LABELS)
    results.extend(prediction_string)
    
    del x, lx, h, lh
    torch.cuda.empty_cache()

In [2]:
data_dir = "/content/11-785-s23-hw3p2" + "/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

!kaggle competitions submit -c 11-785-s23-hw3p2 -f submission.csv -m "I made it!"

In [ ]:
run.finish()